In [1]:
from loader import train_loader, test_loader, valid_loader

In [2]:
from model import UKCONV

In [4]:
UKCONV_model = UKCONV()

UKCONV_model

UKCONV(
  (C1): KAN_Convolutional_Layer(
    (convs): ModuleList(
      (0-15): 16 x KAN_Convolution(
        (conv): KANLinear(
          (base_activation): SiLU()
        )
      )
    )
  )
  (D1): DownSampling(
    (Down): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
  )
  (C2): KAN_Convolutional_Layer(
    (convs): ModuleList(
      (0-31): 32 x KAN_Convolution(
        (conv): KANLinear(
          (base_activation): SiLU()
        )
      )
    )
  )
  (D2): DownSampling(
    (Down): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
    )
  )
  (C3): KAN_Convolutional_Layer(
    (convs): ModuleList(
      (0-63): 64 x KAN_Convolution(
        (conv): KANLinear(
          (base_activation): SiLU()
        )
      )
    )
  )
  (D3): DownSampling(
    (Down): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3,

In [5]:
import torch
import torch.nn as nn

In [6]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [7]:
UKCONV_model=UKCONV_model.to(DEVICE)

loss_fn= nn.BCEWithLogitsLoss()
lr=0.0001
optimizer=torch.optim.Adam(params=UKCONV_model.parameters(), lr=lr)
n_epochs = 10

In [8]:
from tqdm.notebook import tqdm

def train_one_epoch(model, optimizer, loss_fn, dataloader):
    model.train()
    train_loss = 0.0
    for batch in tqdm(dataloader, desc="Training"):
        images, masks = batch
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss /= len(dataloader.dataset)
    return train_loss

In [10]:
for epoch in range(n_epochs):
    train_loss = train_one_epoch(UKCONV_model, optimizer, loss_fn, train_loader)
    print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}")

Training:   0%|          | 0/24 [00:00<?, ?it/s]

: 

In [ ]:
torch.save(UKCONV_model.state_dict(), 'UKCONV_model.pth')

In [ ]:
from model import UKCONV
from torch import nn
import torch

UKCONV_model = UKCONV()
torch.load('UKCONV_model.pth') 

loss_fn= nn.BCEWithLogitsLoss()

In [ ]:
from loader import test_loader

test_loss = 0.0
UKCONV_model.eval()
with torch.no_grad():
    for batch in test_loader:
        images, masks = batch
        outputs = UKCONV_model(images)
        loss = loss_fn(outputs, masks)
        test_loss += loss.item() * images.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")